# 02 — Error Analysis

**Objective**: Diagnose failure modes (FP/FN/confusions) and produce publication-grade error summaries.

Expected inputs:
- prediction CSV/JSON (per image: boxes, class, conf)
- ground-truth labels (YOLO)

Outputs:
- per-class FP/FN counts
- confusion-like summary for detection (IoU-matched)
- curated hard cases for qualitative figures


In [ ]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

DATA_ROOT = Path(os.getenv("DATA_ROOT", "../data"))
OUTPUT_ROOT = Path(os.getenv("OUTPUT_ROOT", "../outputs"))
(OUTPUT_ROOT / "tables").mkdir(parents=True, exist_ok=True)
(OUTPUT_ROOT / "figures").mkdir(parents=True, exist_ok=True)

# Update these paths to your exported predictions
PRED_PATH = OUTPUT_ROOT / "metrics" / "raw" / "predictions.csv"  # placeholder
DATASET_ROOT = DATA_ROOT / "processed" / "D1_yolo640"
IMAGES_DIR = DATASET_ROOT / "images"
LABELS_DIR = DATASET_ROOT / "labels"

CLASSES = [
    "Cleft Lip",
    "Epibulbar Dermoid",
    "Eyelid Coloboma",
    "Facial Asymmetry",
    "Malocclusion",
    "Microtia",
    "Vertebral Abnormalities",
]
NUM_CLASSES = len(CLASSES)


## Prediction file format (recommended)

A simple CSV schema that is easy to analyze:

- `image` (relative path)
- `pred_cls`, `pred_conf`, `pred_x1`, `pred_y1`, `pred_x2`, `pred_y2`  (pixel coords)

You can store multiple predictions per image (one row per predicted box).


In [ ]:
def yolo_to_xyxy(norm_xywh, w, h):
    c, x, y, bw, bh = norm_xywh
    x1 = (x - bw/2) * w
    y1 = (y - bh/2) * h
    x2 = (x + bw/2) * w
    y2 = (y + bh/2) * h
    return int(c), np.array([x1, y1, x2, y2], dtype=float)

def iou_xyxy(a, b):
    ax1, ay1, ax2, ay2 = a
    bx1, by1, bx2, by2 = b
    inter_x1 = max(ax1, bx1)
    inter_y1 = max(ay1, by1)
    inter_x2 = min(ax2, bx2)
    inter_y2 = min(ay2, by2)
    iw = max(0.0, inter_x2 - inter_x1)
    ih = max(0.0, inter_y2 - inter_y1)
    inter = iw * ih
    area_a = max(0.0, ax2-ax1) * max(0.0, ay2-ay1)
    area_b = max(0.0, bx2-bx1) * max(0.0, by2-by1)
    union = area_a + area_b - inter + 1e-12
    return inter / union


In [ ]:
# Load predictions (placeholder)
if not PRED_PATH.exists():
    print("Prediction file not found:", PRED_PATH)
    print("Export predictions to this path or update PRED_PATH.")
else:
    preds = pd.read_csv(PRED_PATH)
    display(preds.head())


## IoU-matched FP/FN analysis (greedy matching)

This uses a simple greedy one-to-one matching per image at a fixed IoU threshold (e.g., 0.5).

For publication, this is mainly used for **diagnostics** and curated cases, while official metrics should come from your evaluation pipeline.


In [ ]:
import cv2

IOU_THR = 0.50
CONF_THR = 0.25

def parse_gt_yolo(label_path: Path, img_w: int, img_h: int):
    if not label_path.exists():
        return []
    txt = label_path.read_text(encoding="utf-8").strip()
    if not txt:
        return []
    gts = []
    for line in txt.splitlines():
        parts = line.split()
        if len(parts) != 5:
            continue
        c = int(float(parts[0]))
        x, y, bw, bh = map(float, parts[1:])
        _, box = yolo_to_xyxy((c, x, y, bw, bh), img_w, img_h)
        gts.append((c, box))
    return gts

def greedy_match(pred_boxes, gt_boxes, iou_thr):
    matched_gt = set()
    matches = []
    for pi, (pc, pconf, pbox) in enumerate(sorted(pred_boxes, key=lambda x: -x[1])):
        best = (-1, 0.0)
        for gi, (gc, gbox) in enumerate(gt_boxes):
            if gi in matched_gt:
                continue
            iou = iou_xyxy(pbox, gbox)
            if iou > best[1]:
                best = (gi, iou)
        gi, best_iou = best
        if gi >= 0 and best_iou >= iou_thr:
            matched_gt.add(gi)
            gc, _ = gt_boxes[gi]
            matches.append((pc, gc, best_iou))
        else:
            matches.append((pc, None, best_iou))  # FP
    fn = [gt_boxes[gi][0] for gi in range(len(gt_boxes)) if gi not in matched_gt]
    return matches, fn


In [ ]:
if PRED_PATH.exists():
    fp_counts = np.zeros(NUM_CLASSES, dtype=int)
    fn_counts = np.zeros(NUM_CLASSES, dtype=int)
    confusions = np.zeros((NUM_CLASSES, NUM_CLASSES), dtype=int)  # pred x true

    for img_rel, grp in preds.groupby("image"):
        img_path = DATASET_ROOT / img_rel
        if not img_path.exists():
            continue
        img = cv2.imread(str(img_path))
        if img is None:
            continue
        h, w = img.shape[:2]

        gt_path = LABELS_DIR / (Path(img_rel).stem + ".txt")
        gts = parse_gt_yolo(gt_path, w, h)

        pred_boxes = []
        for _, r in grp.iterrows():
            if float(r["pred_conf"]) < CONF_THR:
                continue
            pc = int(r["pred_cls"])
            pconf = float(r["pred_conf"])
            pbox = np.array([r["pred_x1"], r["pred_y1"], r["pred_x2"], r["pred_y2"]], dtype=float)
            pred_boxes.append((pc, pconf, pbox))

        matches, fn = greedy_match(pred_boxes, gts, IOU_THR)

        for pc, gc, best_iou in matches:
            if gc is None:
                if 0 <= pc < NUM_CLASSES:
                    fp_counts[pc] += 1
            else:
                if 0 <= pc < NUM_CLASSES and 0 <= gc < NUM_CLASSES:
                    confusions[pc, gc] += 1

        for gc in fn:
            if 0 <= gc < NUM_CLASSES:
                fn_counts[gc] += 1

    err_df = pd.DataFrame({
        "class": CLASSES,
        "FP": fp_counts,
        "FN": fn_counts,
    })
    display(err_df)

    out_csv = OUTPUT_ROOT / "tables" / "error_fp_fn_by_class.csv"
    err_df.to_csv(out_csv, index=False)
    print("Saved:", out_csv)


In [ ]:
# Plot FP/FN bars
if PRED_PATH.exists():
    fig = plt.figure(figsize=(10, 4), dpi=200)
    x = np.arange(NUM_CLASSES)
    plt.bar(x - 0.2, fp_counts, width=0.4, label="FP")
    plt.bar(x + 0.2, fn_counts, width=0.4, label="FN")
    plt.xticks(x, CLASSES, rotation=30, ha="right")
    plt.ylabel("Count")
    plt.title("Error Profile by Class")
    plt.legend()
    plt.tight_layout()
    out_fig = OUTPUT_ROOT / "figures" / "error_profile_fp_fn.png"
    plt.savefig(out_fig, bbox_inches="tight")
    plt.show()
    print("Saved:", out_fig)
